In [ ]:
pip install transformers

In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install evaluate

In [1]:
from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import TFAutoModelForSequenceClassification, AutoModel
from transformers import AutoTokenizer, pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request
import re
import numpy as np

In [2]:
from scipy import stats

In [3]:
from evaluate import load

In [4]:
from sentence_transformers import SentenceTransformer, util
import sentence_transformers

In [5]:
glue_metric = load('glue', 'stsb')

In [6]:
sts_tr = pd.read_csv("/content/stsb-en-test.csv", names = ["Sent1", "Sent2", "Score"])
sts_tr

,Sent1,Sent2,Score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5
...,...,...,...
1374,"Philippines, Canada pledge to further boost re...",Philippines saves 100 after ferry sinks,0.0
1375,Israel bars Palestinians from Jerusalem's Old ...,"Two-state solution between Palestinians, Israe...",1.0
1376,How much do you know about Secret Service?,Lawmakers from both sides express outrage at S...,1.0
1377,Obama Struggles to Soothe Saudi Fears As Iran ...,Myanmar Struggles to Finalize Voter Lists for ...,0.0


In [7]:
sent1 = list(sts_tr["Sent1"])
sent2 = list(sts_tr["Sent2"])
score = list(sts_tr["Score"])

In [8]:
tmp_score = np.array(score)
scaled_score = tmp_score/5
scaled_score.shape

(1379,)

#MPNET V1 Embeddings on STS b Train 

In [9]:
mpnet_v1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [10]:
mpnet_v1_sent1_emb = mpnet_v1.encode(sent1)
mpnet_v1_sent2_emb = mpnet_v1.encode(sent2)

In [11]:
mp_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(mpnet_v1_sent1_emb[i], mpnet_v1_sent2_emb[i])[0].cpu().T
  mp_v1_score.append(tmp)

mp_v1_score = np.array(mp_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2318.)
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(corre

In [12]:
results = glue_metric.compute(predictions=mp_v1_score, references=scaled_score)
results

{'pearson': 0.8392273021754508, 'spearmanr': 0.8301951988370423}

#MPNET V2 Embeddings on STS b Train 

In [13]:
mpnet_v2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [14]:
mpnet_v2_sent1_emb = mpnet_v2.encode(sent1)
mpnet_v2_sent2_emb = mpnet_v2.encode(sent2)

In [15]:
mp_v2_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(mpnet_v2_sent1_emb[i], mpnet_v2_sent2_emb[i])[0].cpu().T
  mp_v2_score.append(tmp)

mp_v2_score = np.array(mp_v2_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [16]:
results = glue_metric.compute(predictions=mp_v2_score, references=scaled_score)
results

{'pearson': 0.8403996152562648, 'spearmanr': 0.8342190421330611}

#All Roberta Large V1 Embeddings on STS b Train 

In [17]:
roberta_lg_v1 = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [18]:
roberta_lg_v1_sent1_emb = roberta_lg_v1.encode(sent1)
roberta_lg_v1_sent2_emb = roberta_lg_v1.encode(sent2)

In [19]:
roberta_lg_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(roberta_lg_v1_sent1_emb[i], roberta_lg_v1_sent2_emb[i])[0].cpu().T
  roberta_lg_v1_score.append(tmp)

roberta_lg_v1_score = np.array(roberta_lg_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [20]:
results = glue_metric.compute(predictions=roberta_lg_v1_score, references=scaled_score)
results

{'pearson': 0.8418684357509671, 'spearmanr': 0.834718282952068}

#All MINI LM L12 V1 Embeddings on STS b Train 

In [21]:
all_MiniLM_L12_v1 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [22]:
all_MiniLM_L12_v1_sent1_emb = all_MiniLM_L12_v1.encode(sent1)
all_MiniLM_L12_v1_sent2_emb = all_MiniLM_L12_v1.encode(sent2)

In [23]:
all_MiniLM_L12_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_MiniLM_L12_v1_sent1_emb[i], all_MiniLM_L12_v1_sent2_emb[i])[0].cpu().T
  all_MiniLM_L12_v1_score.append(tmp)

all_MiniLM_L12_v1_score = np.array(all_MiniLM_L12_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [24]:
results = glue_metric.compute(predictions=all_MiniLM_L12_v1_score, references=scaled_score)
results

{'pearson': 0.8362532761987159, 'spearmanr': 0.8253856836603592}

#MiniLM L6 V2

In [25]:
all_MiniLM_L6_v2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [26]:
all_MiniLM_L6_v2_sent1_emb = all_MiniLM_L6_v2.encode(sent1)
all_MiniLM_L6_v2_sent2_emb = all_MiniLM_L6_v2.encode(sent2)

In [27]:
all_MiniLM_L6_v2_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_MiniLM_L6_v2_sent1_emb[i], all_MiniLM_L6_v2_sent2_emb[i])[0].cpu().T
  all_MiniLM_L6_v2_score.append(tmp)

all_MiniLM_L6_v2_score = np.array(all_MiniLM_L6_v2_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [28]:
results = glue_metric.compute(predictions=all_MiniLM_L6_v2_score, references=scaled_score)
results

{'pearson': 0.8274064175019977, 'spearmanr': 0.8203246731235654}

#All DistilRoberta V1 Embeddings on STS b Train 

In [29]:
all_distilroberta_v1 = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [30]:
all_distilroberta_v1_sent1_emb = all_distilroberta_v1.encode(sent1)
all_distilroberta_v1_sent2_emb = all_distilroberta_v1.encode(sent2)

In [31]:
all_distilroberta_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_distilroberta_v1_sent1_emb[i], all_distilroberta_v1_sent2_emb[i])[0].cpu().T
  all_distilroberta_v1_score.append(tmp)

all_distilroberta_v1_score = np.array(all_distilroberta_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [32]:
results = glue_metric.compute(predictions=all_distilroberta_v1_score, references=scaled_score)
results

{'pearson': 0.8353960220099703, 'spearmanr': 0.8251810884086646}

#AllenAi Specter on STS b train

In [33]:
allen_specter = SentenceTransformer('sentence-transformers/allenai-specter')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

In [34]:
allen_specter_sent1_emb = allen_specter.encode(sent1)
allen_specter_sent2_emb = allen_specter.encode(sent2)

In [35]:
allen_specter_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(allen_specter_sent1_emb[i], allen_specter_sent2_emb[i])[0].cpu().T
  allen_specter_score.append(tmp)

allen_specter_score = np.array(allen_specter_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [36]:
results = glue_metric.compute(predictions=allen_specter_score, references=scaled_score)
results

{'pearson': 0.618573825669309, 'spearmanr': 0.612570795552559}

#Mini LM Ensemble

In [37]:
v_add_sent1 = all_MiniLM_L12_v1_sent1_emb + all_MiniLM_L6_v2_sent1_emb
v_add_sent2 = all_MiniLM_L12_v1_sent2_emb + all_MiniLM_L6_v2_sent2_emb

In [38]:
mini_lm_ensemble_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(v_add_sent1[i], v_add_sent2[i])[0].cpu().T
  mini_lm_ensemble_score.append(tmp)

mini_lm_ensemble_score = np.array(mini_lm_ensemble_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [39]:
results = glue_metric.compute(predictions=mini_lm_ensemble_score, references=scaled_score)
results

{'pearson': 0.8336306246670181, 'spearmanr': 0.8274826458292657}

Horizontal Ensemble

In [40]:
h_add_sent1 = np.concatenate((all_MiniLM_L12_v1_sent1_emb, all_MiniLM_L6_v2_sent1_emb), axis=1)
h_add_sent2 = np.concatenate((all_MiniLM_L12_v1_sent2_emb, all_MiniLM_L6_v2_sent2_emb), axis=1)

In [41]:
h_add_lm_ensemble_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(h_add_sent1[i], h_add_sent2[i])[0].cpu().T
  h_add_lm_ensemble_score.append(tmp)

h_add_lm_ensemble_score = np.array(h_add_lm_ensemble_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [42]:
results = glue_metric.compute(predictions=h_add_lm_ensemble_score, references=scaled_score)
results

{'pearson': 0.8367193216479597, 'spearmanr': 0.8275692050945157}

#Embedding Ensemble MpNET V1- Distill Roberta V1

Vertical Ensemble

In [43]:
v_add_sent1 = mpnet_v1_sent1_emb + all_distilroberta_v1_sent1_emb
v_add_sent2 = mpnet_v1_sent2_emb + all_distilroberta_v1_sent2_emb

In [44]:
v_add_mpnet_distill_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(v_add_sent1[i], v_add_sent2[i])[0].cpu().T
  v_add_mpnet_distill_score.append(tmp)

v_add_mpnet_distill_score = np.array(v_add_mpnet_distill_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [45]:
results = glue_metric.compute(predictions=v_add_mpnet_distill_score, references=scaled_score)
results

{'pearson': 0.8418865492085233, 'spearmanr': 0.8325395374667282}

Horizontal Ensemble

In [46]:
h_add_sent1 = np.concatenate((mpnet_v1_sent1_emb, all_distilroberta_v1_sent1_emb), axis=1)
h_add_sent2 = np.concatenate((mpnet_v1_sent2_emb, all_distilroberta_v1_sent2_emb), axis=1)

In [47]:
h_add_mpnet_distill_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(h_add_sent1[i], h_add_sent2[i])[0].cpu().T
  h_add_mpnet_distill_score.append(tmp)

h_add_mpnet_distill_score = np.array(h_add_mpnet_distill_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [48]:
results = glue_metric.compute(predictions=h_add_mpnet_distill_score, references=scaled_score)
results

{'pearson': 0.8430515632898511, 'spearmanr': 0.8333544482325052}